In [2]:
%matplotlib inline
%load_ext rpy2.ipython

In [3]:
%%R
workDir = '/home/chantal/Chazy/Bulk/515_806/data'
figDir = '/home/chantal/Chazy/Bulk/515_806/data/figs/'

physeqDir = '/home/chantal/Chazy/data/phyloseq/'
physeqBulk = 'bulk-core-rare'
#physeqBulk = 'bulk-core' 

#l2fcResFile = file.path(workDir, 'l2fc_sparse-maxRejH_BULK_0.25_.txt')
## NOTEBOOK RUN WITH 0.25 SPARSITY FILTERED DATASET...

nprocs = 20

In [4]:
%%R
library(ggplot2)
library(phyloseq)
library(gridExtra)
library(data.table)
library(vegan)
library(plyr);library(dplyr)
library(tidyr)
library(picante)

/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: data.table 1.9.6  For help type ?data.table or https://github.com/Rdatatable/data.table/wiki

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: The fastest way to learn (by data.table authors): https://www.datacamp.com/courses/data-analysis-the-data-table-way

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: permute

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: lattice

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: This is vegan 2.3-3

  res =

In [9]:
%%R

##Importing and pruning physeq object

F = file.path(physeqDir, physeqBulk)
physeq.Bulk = readRDS(F)
physeq.Bulk.m = physeq.Bulk %>% sample_data
physeq.Bulk = prune_samples((physeq.Bulk.m$Land_Management != "PAS"),
                           physeq.Bulk)


physeq.Bulk.m = physeq.Bulk %>% sample_data

physeq.Bulk


phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 19331 taxa and 175 samples ]
sample_data() Sample Data:       [ 175 samples by 25 sample variables ]
tax_table()   Taxonomy Table:    [ 19331 taxa by 8 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 19331 tips and 19330 internal nodes ]


In [10]:
%%R
#Creating dataframe from physeq object
mdf = psmelt(physeq.Bulk)

In [11]:
%%R
#Rearranging dataframe to have samples as rows and OTUs as columns (needed for future analyses)
OTU.df = mdf %>% select(OTU, Sample, Abundance) %>% spread(OTU, Abundance)


In [12]:
%%R
#Setting OTUs as rownames
rownames(OTU.df) = OTU.df$Sample
OTU.df$Sample = NULL

In [13]:
%%R
#Normalizing OTU counts 
comm <- decostand(OTU.df, method = "total")
# check total abundance in each sample
#apply(comm, 1, sum)

In [14]:
%%R
# Importing metadata
meta = read.table('/home/chantal/Chazy/data/Metadata_515806_all.txt', header = TRUE, sep = '\t')
length(meta$Full.sample)


[1] 1350


In [15]:
%%R -h 2000
#Importing phylogenetic tree
phy = read.tree('/home/chantal/Chazy/otusn_pick.tree')
#plot(phy, cex = 0.5)

In [16]:
%%R
combined = match.phylo.comm(phy, comm)
# the resulting object is a list with $phy and $comm elements.  replace our
# original data with the sorted/matched data
phy = combined$phy
comm = combined$comm

[1] "Dropping tips from the tree because they are not present in the community data:"
   [1] "X90478"    "OTU.28525" "OTU.21627" "OTU.25663" "OTU.30295" "OTU.29905"
   [7] "OTU.13048" "OTU.28832" "OTU.15717" "OTU.19547" "OTU.25349" "OTU.23194"
  [13] "OTU.30164" "OTU.22637" "OTU.11862" "OTU.27417" "OTU.23191" "OTU.19873"
  [19] "OTU.20887" "OTU.28958" "OTU.23149" "OTU.24288" "OTU.22124" "OTU.30177"
  [25] "OTU.31008" "OTU.30174" "OTU.30568" "OTU.23625" "OTU.27299" "OTU.14409"
  [31] "OTU.26564" "OTU.17766" "OTU.21573" "OTU.20980" "OTU.10870" "OTU.27968"
  [37] "OTU.27317" "OTU.20957" "OTU.23591" "OTU.10697" "OTU.23361" "OTU.14742"
  [43] "OTU.27183" "OTU.18445" "OTU.17136" "OTU.11525" "OTU.23305" "OTU.27009"
  [49] "OTU.30631" "OTU.27444" "OTU.11358" "OTU.24164" "OTU.30744" "OTU.12131"
  [55] "OTU.30499" "OTU.9831"  "OTU.11411" "OTU.26201" "OTU.16985" "OTU.15268"
  [61] "OTU.13459" "OTU.15536" "OTU.19959" "OTU.17550" "OTU.14547" "OTU.22818"
  [67] "OTU.12019" "OTU.9394"  "OTU.16403" "O

## Calculating NRI (SES - to get NRI, multiply z values by -1)

In [21]:
%%R
df.l2fc.Till = read.table('data/l2fc_Till.txt', sep = '\t', header = TRUE)
head(df.l2fc.Till)

  sparsity_threshold       OTU log2FoldChange     lfcSE         padj    Rank1
1               0.25  OTU.1739      0.4012126 0.4523957 4.684148e-01 Bacteria
2               0.25  OTU.1813     -1.4802581 0.4326507 1.826795e-03 Bacteria
3               0.25   OTU.778     -0.5498601 0.1379296 2.483877e-04 Bacteria
4               0.25  OTU.2180     -0.3066501 0.2858391 3.755368e-01 Bacteria
5               0.25 OTU.12365     -0.1443963 0.2567288 6.562578e-01 Bacteria
6               0.25  OTU.2343      1.3595096 0.3116793 5.598476e-05 Bacteria
             Rank2          Rank3                  Rank4 Rank5 Rank6 Rank7
1 __Planctomycetes        __OM190 __uncultured_bacterium  <NA>  <NA>  <NA>
2 __Planctomycetes        __OM190 __uncultured_bacterium  <NA>  <NA>  <NA>
3 __Planctomycetes __Pla4_lineage __uncultured_bacterium  <NA>  <NA>  <NA>
4 __Planctomycetes __Pla4_lineage __uncultured_bacterium  <NA>  <NA>  <NA>
5 __Planctomycetes __Pla4_lineage __uncultured_bacterium  <NA>  <NA>  <NA>
6 __

In [18]:
%%R
N.l2fc = filter(df.l2fc.Till, padj2 <=0.10, Till == 'N')
P.l2fc = filter(df.l2fc.Till, padj2 <=0.10, Till == 'T')

In [22]:
%%R
head(mdf)
N = filter(mdf, OTU %in% N.l2fc$OTU)
P = filter(mdf, OTU %in% P.l2fc$OTU)

In [23]:
%%R
print(length(unique(N$OTU)))
print(length(unique(P$OTU)))


[1] 860
[1] 1152


In [24]:
%%R
OTUs.N = N %>% select(OTU, Sample, Abundance) %>% spread(OTU, Abundance)

OTUs.P = P %>% select(OTU, Sample, Abundance) %>% spread(OTU, Abundance)

In [27]:
%%R
nrow(OTUs.P)

[1] 175


In [30]:
%%R
#Setting OTUs as rownames
rownames(OTUs.N) = OTUs.N$Sample
OTUs.N$Sample = NULL

rownames(OTUs.P) = OTUs.P$Sample
OTUs.P$Sample = NULL

In [31]:
%%R
#Normalizing OTU counts 
comm.N <- decostand(OTUs.N, method = "total")

comm.P <- decostand(OTUs.P, method = "total")

In [32]:
%%R
tree = as.phylo(phy)

In [ ]:
%%R
# For NRI, create 2 comm files - one for NT and one for PT...but use whole tree for both
nri.N = ses.mpd(comm.N,cophenetic(tree),null.model="taxa.labels",runs=1000)


In [ ]:
%%R 
write.csv(nri.N, file = 'data/NRI_N_all.csv')
nri.N = 0 

In [6]:
%%R
NRI_N = read.csv('data/NRI_N_all.csv')
head(NRI_N)
summary(NRI_N)

       X           x    
 Min.   :1   Min.   :0  
 1st Qu.:1   1st Qu.:0  
 Median :1   Median :0  
 Mean   :1   Mean   :0  
 3rd Qu.:1   3rd Qu.:0  
 Max.   :1   Max.   :0  


In [7]:
%%R
head(NRI_N)

  X x
1 1 0


In [ ]:
%%R
NRI_N$treat = 'N'
head(NRI_N)

In [ ]:
%%R 
#it is a bicaudal test, so use p values of 0.025 and 0.975 to know if the observed value is lower or higher 
#than expected by chance
head(nri.N)


In [ ]:
%%R
nri.P = ses.mpd(comm.P,cophenetic(tree),null.model="taxa.labels", runs = 1000)


In [ ]:
%%R
write.csv(nri.P, file = 'data/NRI_P_all.csv')
nri.P = 0 

In [5]:
%%R
NRI_P = read.csv('data/NRI_P_all.csv')
head(NRI_P)
summary(NRI_P)

             X           ntaxa          mpd.obs       mpd.rand.mean   
 April15.NTH.1:  1   Min.   :367.0   Min.   :0.7413   Min.   :0.8679  
 April15.NTH.2:  1   1st Qu.:547.0   1st Qu.:0.7624   1st Qu.:0.8684  
 April15.NTH.3:  1   Median :649.0   Median :0.7723   Median :0.8686  
 April15.NTH.4:  1   Mean   :677.1   Mean   :0.7705   Mean   :0.8686  
 April15.NTR.1:  1   3rd Qu.:818.5   3rd Qu.:0.7782   3rd Qu.:0.8687  
 April15.NTR.2:  1   Max.   :921.0   Max.   :0.7946   Max.   :0.8691  
 (Other)      :169                                                    
  mpd.rand.sd        mpd.obs.rank   mpd.obs.z         mpd.obs.p       
 Min.   :0.009040   Min.   :1     Min.   :-10.530   Min.   :0.000999  
 1st Qu.:0.009921   1st Qu.:1     1st Qu.: -9.345   1st Qu.:0.000999  
 Median :0.011087   Median :1     Median : -8.896   Median :0.000999  
 Mean   :0.011103   Mean   :1     Mean   : -8.868   Mean   :0.000999  
 3rd Qu.:0.012107   3rd Qu.:1     3rd Qu.: -8.389   3rd Qu.:0.000999  
 Max. 

In [ ]:
%%R
NRI_P$treat = 'P'
head(NRI_P)

In [ ]:
%%R
NRI_tot = rbind(NRI_N, NRI_P)
head(NRI_tot)

In [ ]:
%%R
ggplot(data = NRI_tot, aes(x = treat, y = -mpd.obs.z)) +
geom_boxplot() + theme(text = element_text(size = 18)) + xlab('Treatment') + ylab('NRI')

In [ ]:
%%R
t.test(NRI_tot$mpd.obs.z ~ NRI_tot$treat)

## Calculating NTI (SES, multiply z-values by -1 to get NTI)

In [ ]:
%%R
nti.N = ses.mntd(comm.N,cophenetic(tree),null.model="taxa.labels",runs=1000)


In [ ]:
%%R
write.csv(nti.N, file = 'data/NTI_N_all.csv')
nti.N = 0 

In [8]:
%%R
NTI_N = read.csv('data/NTI_N_all.csv')
summary(NTI_N)

             X           ntaxa          mntd.obs       mntd.rand.mean  
 April15.NTH.1:  1   Min.   :340.0   Min.   :0.09207   Min.   :0.1734  
 April15.NTH.2:  1   1st Qu.:445.5   1st Qu.:0.09771   1st Qu.:0.1823  
 April15.NTH.3:  1   Median :541.0   Median :0.10076   Median :0.1932  
 April15.NTH.4:  1   Mean   :538.1   Mean   :0.10146   Mean   :0.1952  
 April15.NTR.1:  1   3rd Qu.:635.0   3rd Qu.:0.10511   3rd Qu.:0.2071  
 April15.NTR.2:  1   Max.   :725.0   Max.   :0.11574   Max.   :0.2280  
 (Other)      :169                                                     
  mntd.rand.sd      mntd.obs.rank   mntd.obs.z       mntd.obs.p      
 Min.   :0.004350   Min.   :1     Min.   :-18.38   Min.   :0.000999  
 1st Qu.:0.004946   1st Qu.:1     1st Qu.:-17.14   1st Qu.:0.000999  
 Median :0.005555   Median :1     Median :-16.41   Median :0.000999  
 Mean   :0.005805   Mean   :1     Mean   :-16.30   Mean   :0.000999  
 3rd Qu.:0.006600   3rd Qu.:1     3rd Qu.:-15.50   3rd Qu.:0.000999  
 Max

In [ ]:
%%R
nti.P = ses.mntd(comm.P,cophenetic(tree),null.model="taxa.labels",runs=1000)


In [ ]:
%%R
write.csv(nti.P, file = 'data/NTI_P_all.csv')
nti.P = 0 

In [11]:
%%R
NTI.P = read.csv('data/NTI_P_all.csv', header = TRUE)
head(NTI.P)

              X ntaxa   mntd.obs mntd.rand.mean mntd.rand.sd mntd.obs.rank
1 April15.NTH.1   606 0.09986137      0.1853611  0.004874744             1
2 April15.NTH.2   634 0.09441128      0.1822667  0.004700252             1
3 April15.NTH.3   566 0.10006461      0.1902089  0.005417427             1
4 April15.NTH.4   578 0.10439955      0.1886957  0.005293292             1
5 April15.NTR.1   556 0.10095892      0.1913028  0.005313180             1
6 April15.NTR.2   454 0.10271029      0.2058106  0.006267762             1
  mntd.obs.z  mntd.obs.p runs
1  -17.53933 0.000999001 1000
2  -18.69165 0.000999001 1000
3  -16.63968 0.000999001 1000
4  -15.92509 0.000999001 1000
5  -17.00374 0.000999001 1000
6  -16.44930 0.000999001 1000


In [12]:
%%R
summary(NTI.P)

             X           ntaxa          mntd.obs       mntd.rand.mean  
 April15.NTH.1:  1   Min.   :367.0   Min.   :0.08790   Min.   :0.1585  
 April15.NTH.2:  1   1st Qu.:547.0   1st Qu.:0.09356   1st Qu.:0.1657  
 April15.NTH.3:  1   Median :649.0   Median :0.09986   Median :0.1808  
 April15.NTH.4:  1   Mean   :677.1   Mean   :0.10000   Mean   :0.1799  
 April15.NTR.1:  1   3rd Qu.:818.5   3rd Qu.:0.10565   3rd Qu.:0.1924  
 April15.NTR.2:  1   Max.   :921.0   Max.   :0.11819   Max.   :0.2220  
 (Other)      :169                                                     
  mntd.rand.sd      mntd.obs.rank   mntd.obs.z       mntd.obs.p      
 Min.   :0.003541   Min.   :1     Min.   :-20.46   Min.   :0.000999  
 1st Qu.:0.003932   1st Qu.:1     1st Qu.:-18.46   1st Qu.:0.000999  
 Median :0.004620   Median :1     Median :-17.05   Median :0.000999  
 Mean   :0.004742   Mean   :1     Mean   :-17.11   Mean   :0.000999  
 3rd Qu.:0.005483   3rd Qu.:1     3rd Qu.:-15.91   3rd Qu.:0.000999  
 Max